In [117]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
import numpy as np

In [2]:
spark = SparkSession.builder.master('local[16]').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/16 12:00:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/16 12:00:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [202]:
path = 'data/mock_data.csv'
df = spark.read.csv(path, header=True)

In [203]:
df.show(1)

+--------------------+--------------------+-------+--------+-----------+--------------------+---------+--------+-------+------+----------+-----------+--------------------+--------+-------+
|         prot_access|        sequ_md5_dig|seq_len|analysis|sign_access|          sign_descr|start_loc|stop_loc|  score|status|      date|iPro_access|          iPro_descr|GO_annot|pathway|
+--------------------+--------------------+-------+--------+-----------+--------------------+---------+--------+-------+------+----------+-----------+--------------------+--------+-------+
|gi|29898682|gb|AA...|92d1264e347e14924...|    547| TIGRFAM|  TIGR03882|cyclo_dehyd_2: ba...|        2|     131|1.6E-21|     T|25-04-2022|  IPR022291|Bacteriocin biosy...|       -|      -|
+--------------------+--------------------+-------+--------+-----------+--------------------+---------+--------+-------+------+----------+-----------+--------------------+--------+-------+
only showing top 1 row



In [204]:
selection = df.select(['prot_access', 'seq_len', 'iPro_access', 'start_loc', 'stop_loc']).filter("iPro_access != '-'")
selection.show(1)

+--------------------+-------+-----------+---------+--------+
|         prot_access|seq_len|iPro_access|start_loc|stop_loc|
+--------------------+-------+-----------+---------+--------+
|gi|29898682|gb|AA...|    547|  IPR022291|        2|     131|
+--------------------+-------+-----------+---------+--------+
only showing top 1 row



In [205]:
# calculate relative difference
computed = selection.withColumn('difference', (col('stop_loc') - col('start_loc')) / col('seq_len')) \
	.select(['prot_access', 'iPro_access', 'difference'])

In [206]:
computed.show(2)

+--------------------+-----------+-------------------+
|         prot_access|iPro_access|         difference|
+--------------------+-----------+-------------------+
|gi|29898682|gb|AA...|  IPR022291|0.23583180987202926|
|gi|29898682|gb|AA...|  IPR027624| 0.7056672760511883|
+--------------------+-----------+-------------------+
only showing top 2 rows



In [207]:
computed.groupBy('prot_access').pivot('iPro_access').count().show(1)

+--------------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+-------

In [208]:
computed.groupBy('prot_access').count().show(3)

+--------------------+-----+
|         prot_access|count|
+--------------------+-----+
|gi|29894993|gb|AA...|    4|
|gi|29898233|gb|AA...|   21|
|gi|29895012|gb|AA...|   13|
+--------------------+-----+
only showing top 3 rows



In [209]:
# computed.groupBy('prot_access').count().collect()

In [210]:
# testing
# vals = ['gi|29894993|gb|AAP08281.1|','gi|29895012|gb|AAP08300.1|']
# test = computed.filter(computed.prot_access.isin(vals))
test = computed

In [211]:
test.show()

+--------------------+-----------+-------------------+
|         prot_access|iPro_access|         difference|
+--------------------+-----------+-------------------+
|gi|29898682|gb|AA...|  IPR022291|0.23583180987202926|
|gi|29898682|gb|AA...|  IPR027624| 0.7056672760511883|
|gi|29898682|gb|AA...|  IPR003776| 0.7093235831809872|
|gi|29898682|gb|AA...|  IPR003776| 0.6215722120658135|
|gi|29898682|gb|AA...|  IPR003776| 0.6215722120658135|
|gi|29894058|gb|AA...|  IPR039420| 0.9871244635193133|
|gi|29894058|gb|AA...|  IPR001867| 0.4248927038626609|
|gi|29894058|gb|AA...|  IPR001867| 0.3261802575107296|
|gi|29894058|gb|AA...|  IPR001867| 0.3261802575107296|
|gi|29894058|gb|AA...|  IPR001867|0.37339055793991416|
|gi|29894058|gb|AA...|  IPR001789| 0.4721030042918455|
|gi|29894058|gb|AA...|  IPR036388|0.45064377682403434|
|gi|29894058|gb|AA...|  IPR011006| 0.7896995708154506|
|gi|29894058|gb|AA...|  IPR001789| 0.4678111587982833|
|gi|29894058|gb|AA...|  IPR001789|0.48497854077253216|
|gi|298985

In [212]:
test.select('iPro_access').distinct().count()

258

In [213]:
test.groupBy('prot_access').pivot('iPro_access').max().show(2)

+--------------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+-------------------+-----------------+---------+---------+---------+---------+

In [259]:
t2 = test.groupBy('prot_access').pivot('iPro_access').max()
t2.show(1)

AnalysisException: Column 'iPro_access' does not exist. Did you mean one of the following? [prot_access, features, maxValue, IPR000014, IPR000086, IPR000160, IPR000182, IPR000223, IPR000297, IPR000398, IPR000456, IPR000515, IPR000522, IPR000551, IPR000620, IPR000700, IPR000794, IPR000845, IPR001014, IPR001029, IPR001048, IPR001119, IPR001173, IPR001261, IPR001296, IPR001387, IPR001412, IPR001451, IPR001492, IPR001555, IPR001570, IPR001613, IPR001697, IPR001789, IPR001790, IPR001867, IPR001986, IPR002218, IPR002347, IPR002363, IPR002376, IPR002491, IPR002502, IPR002547, IPR002901, IPR002933, IPR002937, IPR003018, IPR003325, IPR003416, IPR003439, IPR003593, IPR003594, IPR003646, IPR003660, IPR003661, IPR003776, IPR003817, IPR003869, IPR004089, IPR004090, IPR004358, IPR004360, IPR004394, IPR004416, IPR004495, IPR004607, IPR004636, IPR004646, IPR004647, IPR005117, IPR005467, IPR005754, IPR005814, IPR005881, IPR006066, IPR006067, IPR006073, IPR006175, IPR006264, IPR006424, IPR006439, IPR006

In [215]:
# t2 = t2.withColumn("y", lit("null"))
# t2.show()

In [216]:
def func(row):
	print(row[1:-1])
	ipro_vals = row[1:-1]
	print(ipro_vals)
	return row

In [258]:
# t2.rdd.map(func).toDF().show()

In [ ]:
t2.show()

+--------------------+------------------+-----------------+----------------+-------------------+-------------------+-----------------+------------------+-------------------+
|         prot_access|         IPR002347|        IPR008599|       IPR011283|          IPR020904|          IPR025736|        IPR036291|         IPR041522|          IPR042070|
+--------------------+------------------+-----------------+----------------+-------------------+-------------------+-----------------+------------------+-------------------+
|gi|29894993|gb|AA...|              null|0.353099730458221|            null|               null|0.15363881401617252|             null|0.3045822102425876|0.32345013477088946|
|gi|29895012|gb|AA...|0.7813765182186235|             null|0.97165991902834|0.11336032388663968|               null|0.979757085020243|              null|               null|
+--------------------+------------------+-----------------+----------------+-------------------+-------------------+--------------

In [218]:
cols=t2.columns[1:]

In [219]:
from pyspark.sql.functions import col, greatest, udf, array
from pyspark.sql.types import StringType
from pyspark.sql.functions import when

In [221]:
t3 = t2.withColumn('maxValue', greatest(*[col(x) for x in cols]))
t3.show(3)

+--------------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+---------+---------+---------+---------+---------+------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+-------------------+-----------------+---------+---------+---------+-

In [223]:
col_arr = t3.columns[1:]

In [224]:
def modify_values(r):
	for i in range(len(r[:-1])):
		if r[i] == r[-1] and r[i] > 0.9:
			return col_arr[i]

In [225]:
modify_values_udf = udf(modify_values, StringType())

In [227]:
t4 = t3.withColumn('maxColumn', modify_values_udf(array(col_arr)))
t4.show(3)

+--------------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+------------------+---------+---------+---------+---------+---------+------------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+-------------------+-----------------+---------+---------+---------+-

In [228]:
cols = t4.columns[1:-2]

In [229]:
t5 = t4
for i in cols:
	t5 = t5.withColumn(i, when(col(i) > 0.0, 1).otherwise(0))

In [231]:
t5.show(3)

+--------------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+-------

In [257]:
t5.count()

81

In [233]:
t6 = t5.filter("maxColumn != 'null'")
t6.show(3)

+--------------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+-------

### Whole converter pipeline

In [237]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [240]:
columns_to_use = t6.columns[2:-2]

In [241]:
assembler = VectorAssembler(inputCols=columns_to_use, outputCol='features')
dataframe = assembler.transform(t6)

In [243]:
label_string = StringIndexer(inputCol='maxColumn', outputCol='labelIndex')
dataframe = label_string.fit(dataframe).transform(dataframe)

In [245]:
dataframe.show(1)

+--------------------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+-------

In [248]:
train, test = dataframe.randomSplit([0.8, 0.2], seed=42)
print('Training dataset count: ', str(train.count()))
print('Test dataset count: ', str(test.count()))

Training dataset count:  37
Test dataset count:  9


In [250]:
rf = RandomForestClassifier(featuresCol='features', labelCol='labelIndex')
rfModel = rf.fit(train)

In [251]:
predictions = rfModel.transform(test)

In [255]:
predictions.select(['labelIndex', 'prediction']).show()

+----------+----------+
|labelIndex|prediction|
+----------+----------+
|      28.0|       0.0|
|       8.0|       0.0|
|      19.0|       0.0|
|      23.0|       0.0|
|      40.0|       0.0|
|      34.0|       0.0|
|      31.0|       0.0|
|      17.0|      38.0|
|       7.0|       0.0|
+----------+----------+



In [256]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="labelIndex", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

Accuracy = 0.0
Test Error = 1.0
